In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge

In [3]:
df = pd.read_csv('train.csv',)

In [4]:
df.tail(10)

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
517744,517744,highway,4,0.38,60,night,clear,True,False,afternoon,True,False,1,0.42
517745,517745,highway,4,0.02,35,daylight,clear,True,True,morning,False,False,0,0.02
517746,517746,rural,4,0.53,70,dim,rainy,True,True,evening,False,False,3,0.53
517747,517747,urban,3,0.39,35,dim,foggy,True,True,morning,False,False,0,0.15
517748,517748,rural,2,0.53,70,dim,foggy,True,True,afternoon,True,True,2,0.37
517749,517749,highway,4,0.10,70,daylight,foggy,True,True,afternoon,False,False,2,0.32
517750,517750,rural,4,0.47,35,daylight,rainy,True,True,morning,False,False,1,0.26
517751,517751,urban,4,0.62,25,daylight,foggy,False,False,afternoon,False,True,0,0.19
517752,517752,highway,3,0.63,25,night,clear,True,False,afternoon,True,True,3,0.51
517753,517753,highway,2,0.31,45,dim,rainy,False,True,afternoon,True,True,2,0.22


In [5]:
df1 = pd.read_csv('synthetic_road_accidents_100k.csv')

In [6]:
df.tail(10)

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
517744,517744,highway,4,0.38,60,night,clear,True,False,afternoon,True,False,1,0.42
517745,517745,highway,4,0.02,35,daylight,clear,True,True,morning,False,False,0,0.02
517746,517746,rural,4,0.53,70,dim,rainy,True,True,evening,False,False,3,0.53
517747,517747,urban,3,0.39,35,dim,foggy,True,True,morning,False,False,0,0.15
517748,517748,rural,2,0.53,70,dim,foggy,True,True,afternoon,True,True,2,0.37
517749,517749,highway,4,0.10,70,daylight,foggy,True,True,afternoon,False,False,2,0.32
517750,517750,rural,4,0.47,35,daylight,rainy,True,True,morning,False,False,1,0.26
517751,517751,urban,4,0.62,25,daylight,foggy,False,False,afternoon,False,True,0,0.19
517752,517752,highway,3,0.63,25,night,clear,True,False,afternoon,True,True,3,0.51
517753,517753,highway,2,0.31,45,dim,rainy,False,True,afternoon,True,True,2,0.22


In [7]:
equal_data = df.equals(df1)
print(equal_data)


False


In [8]:
print("comp train data shape: ",df.shape)
print("original data shape: ",df1.shape)

comp train data shape:  (517754, 14)
original data shape:  (100000, 13)


In [9]:
print("describe train data: \n",df.describe())

describe train data: 
                   id      num_lanes      curvature    speed_limit  \
count  517754.000000  517754.000000  517754.000000  517754.000000   
mean   258876.500000       2.491511       0.488719      46.112575   
std    149462.849975       1.120434       0.272563      15.788521   
min         0.000000       1.000000       0.000000      25.000000   
25%    129438.250000       1.000000       0.260000      35.000000   
50%    258876.500000       2.000000       0.510000      45.000000   
75%    388314.750000       3.000000       0.710000      60.000000   
max    517753.000000       4.000000       1.000000      70.000000   

       num_reported_accidents  accident_risk  
count           517754.000000  517754.000000  
mean                 1.187970       0.352377  
std                  0.895961       0.166417  
min                  0.000000       0.000000  
25%                  1.000000       0.230000  
50%                  1.000000       0.340000  
75%                  2.000

In [10]:
print("describe original data: \n",df1.describe())

describe original data: 
            num_lanes      curvature    speed_limit  num_reported_accidents  \
count  100000.000000  100000.000000  100000.000000            100000.00000   
mean        2.495300       0.500619      47.054550                 1.50308   
std         1.117958       0.288691      16.321527                 1.22866   
min         1.000000       0.000000      25.000000                 0.00000   
25%         1.000000       0.250000      35.000000                 1.00000   
50%         2.000000       0.500000      45.000000                 1.00000   
75%         3.000000       0.750000      60.000000                 2.00000   
max         4.000000       1.000000      70.000000                10.00000   

       accident_risk  
count  100000.000000  
mean        0.382613  
std         0.178692  
min         0.000000  
25%         0.250000  
50%         0.380000  
75%         0.510000  
max         1.000000  


In [11]:
df['is_original'] = 0
df1['is_original']=1

In [12]:
df.head(2)

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk,is_original
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13,0
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35,0


In [13]:
df1.head(2)

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk,is_original
0,rural,2,0.29,70,night,rainy,False,True,evening,False,False,1,0.64,1
1,highway,1,0.34,25,dim,clear,False,False,morning,False,False,3,0.27,1


In [14]:
comp_df = pd.concat([df,df1],ignore_index=1)

In [15]:
comp_df.shape

(617754, 15)

In [16]:
comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617754 entries, 0 to 617753
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      517754 non-null  float64
 1   road_type               617754 non-null  object 
 2   num_lanes               617754 non-null  int64  
 3   curvature               617754 non-null  float64
 4   speed_limit             617754 non-null  int64  
 5   lighting                617754 non-null  object 
 6   weather                 617754 non-null  object 
 7   road_signs_present      617754 non-null  bool   
 8   public_road             617754 non-null  bool   
 9   time_of_day             617754 non-null  object 
 10  holiday                 617754 non-null  bool   
 11  school_season           617754 non-null  bool   
 12  num_reported_accidents  617754 non-null  int64  
 13  accident_risk           617754 non-null  float64
 14  is_original         

In [17]:
comp_df.isnull().sum()

id                        100000
road_type                      0
num_lanes                      0
curvature                      0
speed_limit                    0
lighting                       0
weather                        0
road_signs_present             0
public_road                    0
time_of_day                    0
holiday                        0
school_season                  0
num_reported_accidents         0
accident_risk                  0
is_original                    0
dtype: int64

In [18]:
pd.read_csv('sample_submission.csv')

,id,accident_risk
0,517754,0.352
1,517755,0.352
2,517756,0.352
3,517757,0.352
4,517758,0.352
...,...,...
172580,690334,0.352
172581,690335,0.352
172582,690336,0.352
172583,690337,0.352


In [19]:
df_test = pd.read_csv('test.csv')

In [20]:
if 'id' in comp_df.columns:
    full_train_df = comp_df.drop('id', axis=1)

In [22]:
print(comp_df.info)

<bound method DataFrame.info of          id road_type  num_lanes  curvature  speed_limit  lighting weather  \
0       0.0     urban          2       0.06           35  daylight   rainy   
1       1.0     urban          4       0.99           35  daylight   clear   
2       2.0     rural          4       0.63           70       dim   clear   
3       3.0   highway          4       0.07           35       dim   rainy   
4       4.0     rural          1       0.58           60  daylight   foggy   
...     ...       ...        ...        ...          ...       ...     ...   
617749  NaN     rural          2       0.61           60       dim   foggy   
617750  NaN     rural          2       0.52           35       dim   foggy   
617751  NaN     rural          2       0.08           70  daylight   clear   
617752  NaN     rural          1       0.83           70       dim   foggy   
617753  NaN     rural          2       0.01           45  daylight   foggy   

        road_signs_present  pub

In [23]:
comp_df.describe()

,id,num_lanes,curvature,speed_limit,num_reported_accidents,accident_risk,is_original
count,517754.000000,617754.000000,617754.000000,617754.000000,617754.000000,617754.000000,617754.000000
mean,258876.500000,2.492125,0.490645,46.265059,1.238979,0.357272,0.161877
std,149462.849975,1.120034,0.275273,15.879793,0.964697,0.168832,0.368338
min,0.000000,1.000000,0.000000,25.000000,0.000000,0.000000,0.000000
25%,129438.250000,1.000000,0.260000,35.000000,1.000000,0.240000,0.000000
50%,258876.500000,2.000000,0.510000,45.000000,1.000000,0.350000,0.000000
75%,388314.750000,3.000000,0.720000,60.000000,2.000000,0.470000,0.000000
max,517753.000000,4.000000,1.000000,70.000000,10.000000,1.000000,1.000000


In [24]:
comp_df.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk,is_original
0,0.0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13,0
1,1.0,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35,0
2,2.0,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30,0
3,3.0,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21,0
4,4.0,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56,0


In [25]:
comp_df.columns

Index(['id', 'road_type', 'num_lanes', 'curvature', 'speed_limit', 'lighting',
       'weather', 'road_signs_present', 'public_road', 'time_of_day',
       'holiday', 'school_season', 'num_reported_accidents', 'accident_risk',
       'is_original'],
      dtype='object')